import logging

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import faiss

INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In [3]:
index = faiss.IndexFlatL2(1024)

In [4]:
from bedrock_fm import from_model_id, Model
import boto3
sess = boto3.Session(region_name='us-west-2')

emb = from_model_id(Model.COHERE_EMBED_MULTILINGUAL_V3, session=sess)

In [5]:
import json
with open("apts_db.json") as f:
    apts_db = json.loads(f.read())

In [6]:
passages = []
for i in range(0,1000):
    description = apts_db[i]['description']
    for p in description.split('.'):
        passages.append({"passage": p, "index": i})
    

In [7]:
len(passages)

12017

In [24]:
import numpy as np
embeddings = []
step = 10
for i in range(0,len(passages),step):
    ps = [p["passage"] for p in passages[i:i+step]]
    v = emb.generate(ps)
    embeddings = embeddings + [np.array(x).astype(np.float32) for x in v]
    # index.add(np.array(v).astype(np.float32))


In [9]:
index.is_trained
for e in embeddings:
     index.add(e.reshape(1,1024))

In [8]:

from bedrock_fm import Claude
fm = Claude.from_id(Model.ANTHROPIC_CLAUDE_V2, session=sess)

In [10]:
query="appartamento da ristrutturare con 2 bagni, doppia esposizione e grande balcone o terrazza"

In [11]:
embq = emb.generate_for_query(query)

In [14]:
len(embq)

1024

In [22]:
ref = fm.generate("Utilizza la domanda nei tag <domanda></domanda> per creare la descrizione di un appartamento in vendita a Milano. Genera la risposta nei tags <risposta></risposta>.\n<domanda>{query}</domanda>\n".format(query = query))


INFO:bedrock_fm.bedrock:Body= {"prompt": "\n\nHuman:Utilizza la domanda nei tag <domanda></domanda> per creare la descrizione di un appartamento in vendita a Milano. Genera la risposta nei tags <risposta></risposta>.\n<domanda>appartamento da ristrutturare con 2 bagni, doppia esposizione e grande balcone o terrazza</domanda>\n\n\nAssistant:", "max_tokens_to_sample": 500, "stop_sequences": ["\n\nHuman:"], "temperature": 0.7, "top_p": 1, "anthropic_version": "bedrock-2023-05-31"}


In [23]:
s = emb.generate(ref)

In [17]:
sr = index.search(np.array(embq).astype(np.float32).reshape(1,1024),20)
sr

(array([[0.72610015, 0.7288239 , 0.73393834, 0.7600924 , 0.76362514,
         0.7679779 , 0.77264416, 0.7825053 , 0.7912174 , 0.7912174 ,
         0.79277676, 0.81091666, 0.81091666, 0.8414112 , 0.8453514 ,
         0.8561789 , 0.871074  , 0.8906592 , 0.9039991 , 0.9151278 ]],
       dtype=float32),
 array([[79, 77, 69, 53, 72, 31, 29, 38, 16, 22, 83, 17, 23, 89, 11, 59,
         85, 39, 91, 90]]))

In [21]:
seen = set()
for s in sr[1][0]:
    ds = passages[s]['index']
    if ds in seen:
        continue
    seen.add(ds)
    print(passages[s]['passage'], "\n"*3,apts_db[ds]["description"], '\n\n', f"Price {apts_db[ds]['realEstate']['properties'][0]['price']['value']}")
    print("-"*10)

in una  via tranquilla, adiacente  piazza argentina, in stabile d'epoca signorile, al terzo piano, con portineria ed ascensore, vendesi appartamento totalmente da ristrutturare, composto da ingresso, soggiorno con cucina a vista, camera matrimoniale con cabina armadio, camera singola, 2 bagni piu' bagno ospiti, tre balconi e cantina 


 in una  via tranquilla, adiacente  piazza argentina, in stabile d'epoca signorile, al terzo piano, con portineria ed ascensore, vendesi appartamento totalmente da ristrutturare, composto da ingresso, soggiorno con cucina a vista, camera matrimoniale con cabina armadio, camera singola, 2 bagni piu' bagno ospiti, tre balconi e cantina. 

 Price 590000
----------
Composto da ingresso, ampio soggiorno con accesso a  terrazzo di 10 mq cucina a vista, 3 camere, 2 bagniAppena ristrutturato 


 [Viale Corsica] Luminoso appartamento, 7° piano, 115 mq, elegante condominio d’epoca, con servizio di portineria diurno e ascensore.Composto da ingresso, ampio soggiorno

TypeError: 'IndexFlatL2' object is not subscriptable

In [49]:
import numpy as np

In [57]:
a = np.array(v).astype(dtype=np.float32)

In [58]:
a.shape

(5, 1024)

In [59]:
a.dtype

dtype('float32')

In [22]:
index.codes

<faiss.swigfaiss.UInt8Vector; proxy of <Swig Object of type 'std::vector< unsigned char > *' at 0x169604810> >

In [23]:
faiss.write_index(index, "immo.index")